In [ ]:
pip install transfomers

ERROR: Could not find a version that satisfies the requirement transfomers (from versions: none)
ERROR: No matching distribution found for transfomers


In [ ]:
import pandas as pd
from transformers import BartForConditionalGeneration, BartTokenizer

# Load the CSV file
file_path = "/content/data_with_summaries (1).csv"
df = pd.read_csv(file_path)

# Specify the column to summarize (adjust 'text_column' to match your data)
text_column = "Cleaned Article Text"  # Replace with the actual column name in your CSV
if text_column not in df.columns:
    raise ValueError(f"Column '{text_column}' not found in the CSV file.")

# Initialize the BART model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Function to summarize text
def summarize_text(text, max_length=130, min_length=30):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs["input_ids"], max_length=max_length, min_length=min_length,
                                 length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Apply summarization to each row
df["summary"] = df[text_column].apply(lambda x: summarize_text(x) if isinstance(x, str) else "")

# Save the output to a new CSV file
output_path = "/content/data_with_summaries (1).csv"
df.to_csv(output_path, index=False)

print(f"Summarized file saved to: {output_path}")





Summarized file saved to: /content/data_with_summaries (1).csv


In [ ]:
pip install rouge-score nltk pandas

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=06c833550354acd787926e3f603b1ce1883aa487340b46d455f82f79fb1e119a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
import pandas as pd

# Load the datasets
data = pd.read_csv("/content/summaries.csv")
df = pd.read_csv("/content/rouge_scores.csv")

# Concatenate the 'summary_bart' column from 'data' with 'df'
if 'summary_bart' in data.columns:
    df = pd.concat([df, data['summary_bart']], axis=1)
    print("Data concatenated successfully!")
else:
    raise ValueError("Column 'summary_bart' not found in the dataset 'summaries.csv'.")

# Save the updated DataFrame
df.to_csv("/content/updated_rouge_scores.csv", index=False)
print("Updated DataFrame saved to '/content/updated_rouge_scores.csv'.")


Data concatenated successfully!
Updated DataFrame saved to '/content/updated_rouge_scores.csv'.


In [ ]:
import pandas as pd
from rouge_score import rouge_scorer

# Load the dataset
reference_file_path = "/content/updated_rouge_scores.csv"  # Replace with your file path
data = pd.read_csv(reference_file_path)


# Define the reference column and T5-generated summary columns
reference_column = "Summary"  # Reference summaries
generated_columns = ["summary_bart"]

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Function to calculate ROUGE scores
def calculate_rouge(reference, generated):
    """
    Calculate ROUGE-1, ROUGE-2, and ROUGE-L scores.
    """
    if pd.isnull(reference) or pd.isnull(generated):
        return 0.0, 0.0, 0.0  # Handle missing values gracefully

    scores = scorer.score(reference, generated)
    rouge1 = scores['rouge1'].fmeasure
    rouge2 = scores['rouge2'].fmeasure
    rougeL = scores['rougeL'].fmeasure

    return rouge1, rouge2, rougeL

# Process each T5 model column
for gen_col in generated_columns:
    if gen_col in data.columns:
        print(f"Calculating ROUGE scores for {gen_col}")
        scores = data.apply(lambda row: calculate_rouge(row[reference_column], row[gen_col]), axis=1)

        # Add new columns for each ROUGE score
        data[f'ROUGE-1_{gen_col}'] = scores.apply(lambda x: x[0])
        data[f'ROUGE-2_{gen_col}'] = scores.apply(lambda x: x[1])
        data[f'ROUGE-L_{gen_col}'] = scores.apply(lambda x: x[2])
    else:
        print(f"Column {gen_col} not found in the dataset.")

# Save the results to a new CSV file
output_file_path = "/content/updated_rouge_scores.csv"
data.to_csv(output_file_path, index=False)

print(f"ROUGE scores saved to: {output_file_path}")

Calculating ROUGE scores for summary_bart
ROUGE scores saved to: /content/updated_rouge_scores.csv


In [1]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Specify model and tokenizer
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Define save paths
model_save_path = '/content/bart-large-cnn-model'
tokenizer_save_path = '/content/bart-large-cnn-tokenizer'

# Save model and tokenizer
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

print(f"Model saved to: {model_save_path}")
print(f"Tokenizer saved to: {tokenizer_save_path}")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Model saved to: /content/bart-large-cnn-model
Tokenizer saved to: /content/bart-large-cnn-tokenizer
